## Retrieval

First we have to load the corpus.

In [1]:
from datasets import load_dataset
meta_corpus = load_dataset(
    "json",
    data_files="data/corpus_chunks.jsonl",
    split="train"
).to_list()

In [2]:
import copy
import unicodedata as ud
import re
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm
import string

def split_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.lower().split()
    words = [word for word in words if len(word.strip()) > 0]
    return words

## initiate BM25 retriever
tokenized_corpus = [split_text(doc["passage"]) for doc in tqdm(meta_corpus)]
bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/48532 [00:00<?, ?it/s]

In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import pickle
from pyvi.ViTokenizer import tokenize
import numpy as np
from tqdm.notebook import tqdm 

## initiate semantic rertiever
with open('data/corpus_embedding_w150.pkl', 'rb') as f:
    corpus_embs = pickle.load(f)

embedder = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder').cuda()

To improve from simple single BM25 retriever, we now incorporate a sentence transformer model to do the semantic search. \
We will use `bkai-foundation-models/vietnamese-bi-encoder` model which supports Vietnamese pretty well.

In [4]:
from copy import deepcopy
def retrieve(question, topk=50):
    """
    Get most relevant chunks to the question using combination of BM25 and semantic scores.
    """
    ## initialize query for each retriever (BM25 and semantic)
    tokenized_query = split_text(question)
    segmented_question = tokenize(question)
    question_emb = embedder.encode([segmented_question])
    question_emb /= np.linalg.norm(question_emb, axis=1)[:, np.newaxis]

    ## get BM25 and semantic scores
    bm25_scores = bm25.get_scores(tokenized_query)
    semantic_scores = question_emb @ corpus_embs.T
    semantic_scores = semantic_scores[0]

    ## update chunks' scores. 
    max_bm25_score = max(bm25_scores)
    min_bm25_score = min(bm25_scores)
    def normalize(x):
        return (x - min_bm25_score + 0.1) / \
        (max_bm25_score - min_bm25_score + 0.1)
        
    corpus_size = len(meta_corpus)
    for i in range(corpus_size):
        meta_corpus[i]["bm25_score"] = bm25_scores[i]
        meta_corpus[i]["bm25_normed_score"] = normalize(bm25_scores[i])
        meta_corpus[i]["semantic_score"] = semantic_scores[i]

    ## compute combined score (BM25 + semantic)
    for passage in meta_corpus:
        passage["combined_score"] = passage["bm25_normed_score"] * 0.4 + \
                                    passage["semantic_score"] * 0.6

    ## sort passages by the combined score
    sorted_passages = sorted(meta_corpus, key=lambda x: x["combined_score"], reverse=True)
    return sorted_passages[:topk]

## Smoothing contexts

Using combination of BM25 and semantic score may still not yield the best result (because of the reasons you will see at the end of this section) \
We can do better by applying several techniques to form suitable contexts to feed to the LLM. \
The following blocs will implement some utility functions with such techniques.

In [5]:
from copy import deepcopy
from underthesea import sent_tokenize
def extract_consecutive_subarray(numbers):
    subarrays = []
    current_subarray = []
    for num in numbers:
        if not current_subarray or num == current_subarray[-1] + 1:
            current_subarray.append(num)
        else:
            subarrays.append(current_subarray)
            current_subarray = [num]

    subarrays.append(current_subarray)  # Append the last subarray
    return subarrays
    
def merge_contexts(passages):
    passages_sorted_by_id = sorted(passages, key=lambda x: x["id"], reverse=False)
    # psg_texts = [x["passage"].strip("Title: ").strip(x["title"]).strip() 
    #              for x in passages_sorted_by_id]
    
    psg_ids = [x["id"] for x in passages_sorted_by_id]
    consecutive_ids = extract_consecutive_subarray(psg_ids)

    merged_contexts = []
    consecutive_psgs = []
    b = 0
    for ids in consecutive_ids:
        psgs = passages_sorted_by_id[b:b+len(ids)]
        psg_texts = [x["passage"].strip("Title: ").strip(x["title"]).strip() 
                     for x in psgs]
        merged = f"Title: {psgs[0]['title']}\n\n" + " ".join(psg_texts)
        b = b+len(ids)
        merged_contexts.append(dict(
            title=psgs[0]['title'], 
            passage=merged,
            score=max([x["combined_score"] for x in psgs]),
            merged_from_ids=ids
        ))
    return merged_contexts

def discard_contexts(passages):
    sorted_passages = sorted(passages, key=lambda x: x["score"], reverse=False)
    if len(sorted_passages) == 1:
        return sorted_passages
    else:
        shortened = deepcopy(sorted_passages)
        for i in range(len(sorted_passages) - 1):
            current, next = sorted_passages[i], sorted_passages[i+1]
            if next["score"] - current["score"] >= 0.05:
                shortened = sorted_passages[i+1:]
        return shortened

def expand_context(passage, word_window=60, n_sent=3):
    # psg_id = passage["id"]
    merged_from_ids = passage["merged_from_ids"]
    title = passage["title"]
    prev_id = merged_from_ids[0] - 1
    next_id = merged_from_ids[-1] + 1
    strip_title = lambda x: x["passage"].strip(f"Title: {x['title']}\n\n")
    
    texts = []
    if prev_id in range(0, len(meta_corpus)):
        prev_psg = meta_corpus[prev_id]
        if prev_psg["title"] == title: 
            prev_text = strip_title(prev_psg)
            # prev_text = " ".join(prev_text.split()[-word_window:])
            prev_text = " ".join(sent_tokenize(prev_text)[-n_sent:])
            texts.append(prev_text)
            
    texts.append(strip_title(passage))
    
    if next_id in range(0, len(meta_corpus)):
        next_psg = meta_corpus[next_id]
        if next_psg["title"] == title: 
            next_text = strip_title(next_psg)
            # next_text = " ".join(next_text.split()[:word_window])
            next_text = " ".join(sent_tokenize(next_text)[:n_sent])
            texts.append(next_text)

    expanded_text = " ".join(texts)
    expanded_text = f"Title: {title}\n{expanded_text}"
    new_passage = deepcopy(passage)
    new_passage["passage"] = expanded_text
    return new_passage

def expand_contexts(passages, word_window=60, n_sent=3):
    new_passages = [expand_context(passage) for passage in passages]
    return new_passages
    
def collapse(passages):
    new_passages = deepcopy(passages)
    titles = {}
    for passage in new_passages:
        title = passage["title"]
        if not titles.get(title):
            titles[title] = [passage]
        else:
            titles[title].append(passage)
    best_passages = []
    for k, v in titles.items():
        best_passage = max(v, key= lambda x: x["score"])
        best_passages.append(best_passage)
    return best_passages

Note that, with our current chunking strategy, each chunk is a passage of exact 150 words (separated by space), not a comprehensive paragraph. The following function will transform retrieved chunks into whole paragraphs. This function also does some heuristics to expand the context window and discard seem-to-be irrelevant contexts.

In [6]:
def smooth_contexts(passages):
    """Make the context fed to the LLM better.
    Args:
        passages (list): Chunks retrieved from BM25 + semantic retrieval. 
        
    Returns:
        list: List of whole paragraphs, usually will be more relevant to the initital question.
    """
    # 1. If consecutive chunks are rertieved, merge them into one big chunk to ensure the continuity.
    merged_contexts = merge_contexts(passages)
    # 2. A heuristic to discard irrelevevant contexts. 
    # It seems to be better to only keep what are elevant so that the model can focus.
    # Also this reduce #tokens LLM has to read.
    shortlisted_contexts = discard_contexts(merged_contexts)
    # 3. Another heuristic. this step is to take advantage of long context understanding of the LLM.
    # In many cases, the retrieved passages are just consecutive words, not a comprehensive paragraph.
    # This is to expand the passage to the whole paragraph that surrounds it. 
    # My intuition about this is that whole paragraph will add necessary and relevant information.
    expanded_contexts = expand_contexts(shortlisted_contexts)
    # 4. Now after all the merging and expanding, if what are left for us is more than one paragraphs
    # from the same wiki page, then we will only take paragraph with highest retrieval score.
    collapsed_contexts = collapse(expanded_contexts)
    return collapsed_contexts

In [7]:
## I encourage you to investigate each smoothing step using the following example
## to understand the benefit of them. 
## You will see that after each step, we will obtain "better" contexts.

question = "Hồ Chí Minh là ai?"
top_passages = retrieve(question, topk=3)
merged_contexts = merge_contexts(top_passages)
shortlisted_contexts = discard_contexts(merged_contexts)
expanded_contexts = expand_contexts(shortlisted_contexts)
collapsed_contexts = collapse(expanded_contexts)

In [8]:
## Uncomment each of these variable to see the differences.

# top_passages
# merged_contexts
# shortlisted_contexts
# expanded_contexts
collapsed_contexts

[{'title': 'Gia đình Hồ Chí Minh',
  'passage': 'Title: Gia đình Hồ Chí Minh\n, tên thật là Nguyễn Sinh Cung, vị chủ tịch đầu tiên của nước Việt Nam Dân chủ Cộng hòa (nay là Cộng hòa xã hội chủ nghĩa Việt Nam), sinh ra trong một gia đình nhà Nho ở làng Sen (hay làng Kim Liên), xã Kim Liên, huyện Nam Đàn, tỉnh Nghệ An. Thân sinh. Nguyễn Sinh Sắc. Nguyễn Sinh Sắc (còn gọi là Nguyễn Sinh Huy, người dân còn gọi là Cụ Phó bảng; 1862 – 1929) là cha của Hồ Chí Minh. Ông là con của ông Nguyễn Sinh Nhậm và bà Hà Thị Hy. Lớn lên trong một môi trường Nho học dưới sự nuôi dạy của nhà Nho và cha vợ của mình là cụ Hoàng Xuân Đường, ông đỗ cử nhân năm 1894 và Phó bảng năm 1901. Năm 1906, ông được triều đình bổ nhiệm chức Thừa biện bộ Lễ; năm 1909, ông nhậm chức Tri huyện Bình Khê tỉnh Bình Định. Làm quan được ít lâu thì bị triều đình thải hồi vì một "tên cường hào" bị ông bắt giam rồi được thả. Sau đó ông đi vào miền Nam và sinh sống tại Làng Hòa An, Cao Lãnh, Đồng Tháp (nay là xã Hòa An, thành phố C

## Generate

In [9]:
prompt_template = (
    "### System:\n"
    "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n\n\n"
    "### Instruction:\n{instruction}\n\n"
    "### Input:\n{input}\n\n"
    "### Response:\n{output}"
)

def get_prompt(question, contexts):
    context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" for i, x in enumerate(contexts)])
    instruction = 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.'
    # instruction = 'As an intelligent AI model, your task is to analyze and integrate information from multiple contexts given below in order to answer questions and provide citations.'
    input = f"Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.\n\n{context}\n\nQuestion: {question}\nHãy trả lời chi tiết và đầy đủ."
    prompt = prompt_template.format(
        instruction=instruction,
        input=input,
        output=''
    )
    return prompt

In [10]:
## Let's see how a prompt fed to the LLM looks like
question = "Hồ Chí Minh là ai?"
top_passages = retrieve(question, topk=3)
smoothed_contexts = smooth_contexts(top_passages)
prompt = get_prompt(question, smoothed_contexts)
print(prompt)

### System:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.



### Instruction:
You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.

### Input:
Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.

Context [1]: Title: Gia đình Hồ Chí Minh
, tên thật là Nguyễn Sinh Cung, vị chủ tịch đầu tiên của nước Việt Nam Dân chủ Cộng hòa (nay là Cộng hòa xã hội chủ nghĩa Việt Nam), sinh ra trong một gia đình nhà Nho ở làng Sen (hay làng Kim Liên), xã Kim Liên, huyện Nam Đàn, tỉnh Nghệ An. Thân sinh. Nguyễn Sinh Sắc. Nguyễn Sinh Sắc (còn gọi là Nguyễn Sinh Huy, người dân còn gọi là Cụ Phó bảng; 1862 – 1929) là cha của Hồ Chí Minh. Ông là con của ông Nguyễn Sinh Nhậm và bà Hà Thị Hy. Lớn lên trong một môi trường Nho học dưới sự nuôi dạy của nhà Nho và cha vợ của mình là cụ Hoàng Xuân Đường, ông đỗ cử nhân năm 1894 và 

In [11]:
from transformers import GenerationConfig, TextStreamer
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig
import torch

torch_dtype = torch.bfloat16
model_id = "llm4fun/vietrag-7b-v1.0"
device = "cuda"

tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    config=LlamaConfig.from_pretrained(model_id),
    torch_dtype=torch_dtype
)

model = model.eval().to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
def generate(prompt, max_new_tokens=1024):
    """Text completion with a given prompt. In other words, give answer to your question.
    Args:
        prompt (str): Basically <instruction> + <question> + <retrieved_context>
        generation_config (not existed yet): For now, please manually tweak hyperparameters
        for generation in the `generation_config` below. Uncomment necessary arguments as you wish.
    Returns:
        list: an answer to the question within the prompt.
    """
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    model.eval()
    with torch.no_grad():
        generation_config = GenerationConfig(
            repetition_penalty=1.13,
            max_new_tokens=max_new_tokens,
            # temperature=0.2,
            # top_p=0.95,
            # top_k=20,
            # bos_token_id=tokenizer.bos_token_id,
            # eos_token_id=tokenizer.eos_token_id,
            # eos_token_id=0, # for open-end generation.
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True,
            return_dict_in_generate=True,
            output_attentions=False,
            output_hidden_states=False,
            output_scores=False,
        )
        streamer = TextStreamer(tokenizer, skip_prompt=True)
        generated = model.generate(
            inputs=input_ids,
            generation_config=generation_config,
            streamer=streamer,
        )

    gen_tokens = generated["sequences"].cpu()[:, len(input_ids[0]):]
    output = tokenizer.batch_decode(gen_tokens)[0]
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

In [13]:
## Let's test what the LLM would generate given a question and its context via a prompt.
output = generate(prompt)

Hồ Chí Minh là một nhà lãnh đạo và chính trị gia người Việt Nam, sinh ra với tên khai sinh là Nguyễn Sinh Cung vào ngày 19 tháng 5 năm 1890 tại làng Sen (hay làng Kim Liên), xã Kim Liên, huyện Nam Đàn, tỉnh Nghệ An. Ông là con trai của Nguyễn Sinh Sắc (còn gọi là Nguyễn Sinh Huy hoặc Cụ Phó bảng) và Hoàng Thị Loan. Cha ông là một nhà Nho và là một quan chức triều đình, trong khi mẹ ông là một người phụ nữ có trách nhiệm và có trách nhiệm. Hồ Chí Minh đã được nuôi dưỡng trong một môi trường Nho học và đã đỗ cử nhân vào năm 1894 và Phó bảng vào năm 1901.</s>


Not the best but pretty accurate.

## End-to-End RAG

It's almost done. Now let's try a simple RAG pipeline with our Wikipedia corpus.

In [14]:
def rag(question, topk=3):
    top_passages = retrieve(question, topk=topk)
    smoothed_contexts = smooth_contexts(top_passages)
    retrieved_context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" 
                    for i, x in enumerate(smoothed_contexts)])
    prompt = get_prompt(question, smoothed_contexts)
    output = generate(prompt)
    result = {
        "retrieved_context": retrieved_context,
        "generated_answer": output
    }
    return result

In [15]:
questions = [
    "Thành phố nào là thủ đô của Việt Nam?",
    "Samsung do ai sở hữu?",
    "Việt Nam có những ca sĩ KPOP nào?",
]
for question in questions:
    print(f"Câu hỏi: {question}")
    output = rag(question)
    print("---" * 30)

Câu hỏi: Thành phố nào là thủ đô của Việt Nam?
Thành phố thủ đô của Việt Nam là Hà Nội. Nó được xếp hạng là đô thị đặc biệt và là trung tâm tổng hợp cấp quốc gia, cấp vùng hoặc cấp tỉnh về kinh tế, tài chính, văn hóa, giáo dục, đào tạo, du lịch, y tế, khoa học và công nghệ, đầu mối giao thông, giao lưu trong nước và quốc tế [1].</s>
------------------------------------------------------------------------------------------
Câu hỏi: Samsung do ai sở hữu?
Samsung Electronics Co., Ltd. hiện được sở hữu hoàn toàn bởi Tập đoàn Samsung Group, một conglomerate kinh doanh toàn diện có trụ sở tại Hàn Quốc. Tập đoàn Samsung Group được thành lập bởi Lee Byung-Chull, người đã thành lập Công ty Samsung vào năm 1938. Sau khi ông qua đời vào năm 1987, con trai ông, Lee Kun-Hee, đã tiếp quản vai trò lãnh đạo của công ty và đã dẫn dắt công ty trở thành một trong những công ty lớn nhất thế giới. Ngày nay, Samsung Electronics Co., Ltd. là một trong những công ty lớn nhất thế giới, với các sản phẩm và dịch